In [2]:
import warnings
warnings.simplefilter("ignore")
import pickle
import pandas as pd
import numpy as np
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
import time
from tqdm import tqdm_notebook
from collections import namedtuple

### Загрузка таблицы с адресами

In [1]:
SEARCHING_MAP = {'Еда': 'eat',
                      'Авто': 'auto',
                      'Покупки': 'purchase',
                      'Красота': 'beauty',
                      'Услуги': 'service',
                      'Развлечения и отдых': 'entertainment',
                      'Спорт': 'sport',
                      'Религия': 'religion', 
                      'Транспорт': 'transport',
                      'Госуслуги': 'government',
                      'Деньги': 'money',
                      'Образование': 'education',
                      'Стройка и ремонт': 'repair',
                      'Путешествия': 'travel'}

In [2]:
SEARCHING_MAP.values()

dict_values(['eat', 'auto', 'purchase', 'beauty', 'service', 'entertainment', 'sport', 'religion', 'transport', 'government', 'money', 'education', 'repair', 'travel'])

In [27]:
Station = namedtuple('Station', 'name distance')
Organization = namedtuple('Organization', 'name rating reviews mean_check categories address')

In [28]:
class ImportantObject():
    def __init__(self, name, rating, reviews, mean_check, categories, link,  address):
        self.name = name
        self.rating = rating # "Народная" оценка уровня ослуживания 
        self.reviews = reviews
        self.lat, self.long = None, None # Широта и долгота
        self.categories = categories  # Типы объекта по внутренней классификации Яндекс.Карт
        self.link = link # Ссылка на объект 
        self.address = address  # Адрес  
        
    def __str__(self):
        return ("name = {name}, /n"
                "rating = {rating}, /n"
                "reviews = {reviews},/n"
                "lat = {lat}, /n"
                "long = {long}, /n" 
                "categories = {categories} /n"  
                "link = {link} /n"
                "address = {address} /n").format(name= self.name,
                                                rating = self.rating,
                                                reviews = self.reviews,
                                                lat = self.lat,
                                                long = self.long,
                                                categories = self.categories,
                                                link = self.link,
                                                address = self.address)

In [39]:
def parse_inbuilding(driver, atm):
    # Переход на страницу содержащую список объектов находящихся в здании с банкоматом если обектов много
    try:
        try: 
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH,"//div[@class='businesses-list-view__hint _interactive']")))
            sections = driver.find_elements_by_xpath("//div[@class='businesses-list-view__hint _interactive']")
            sections[0].click()
            #print('Длинный список организаций')
            time.sleep(1)
            elements_org = get_elements(driver)
            # Записываем результаты в объект
            atm.inbuilding = [parse_one_organization(element) for element in elements_org] 
            # возвращение на исходную станицу 
            driver.find_elements_by_xpath("//div[@class='sidebar-panel-header-back-view__icon']")[0].click()
            time.sleep(2)
        except: # если объектов меньше 5
            #print('Короткий список организаций')
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH,"//div[@class='businesses-list-view__list']/div")))
            count_org = len(driver.find_elements_by_xpath("//div[@class='businesses-list-view__list']/div"))
            for i in range(count_org):
                WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH,"//div[@class='businesses-list-view__list']/div")))
                elements = driver.find_elements_by_xpath("//div[@class='businesses-list-view__list']/div")
                action=ActionChains(driver)
                action.move_to_element(elements[i]).click().perform() 
                # elements[i].location_once_scrolled_into_view
                # elements[i].click()
                # time.sleep(2)
                atm.inbuilding.append(parsing_business_card_view(driver))
                # возвращение на исходную станицу 
                driver.find_elements_by_xpath("//div[@class='sidebar-panel-header-back-view__icon']")[0].click()
    except: 
        pass

def parsing_business_card_view(element):
        name, rating, reviews, mean_check, categories, link, address = [None]*7
        # print('Заведедние в здании')
        try:
            WebDriverWait(element, 10).until(EC.element_to_be_clickable((By.XPATH,".//div[@class='card-title-view__title-link']")))
        except:
            pass
            # print('Не загрузилась страница')
        try:
            name = element.find_element_by_xpath(".//div[@class='card-title-view__title-link']").text
            # name = element.find_elements_by_xpath(".//div[@class='card-title-view__title']/a[1]")[0].text
        except: pass 
        
        try:
            rating = element.find_elements_by_xpath(".//span[@class='business-rating-badge-view__rating']")[0].text
        except: pass
        
        try:
            reviews = element.find_elements_by_xpath(".//span[@class='business-reviews-count-view']")[0].text
        except: pass

        try:
            mean_check = element.find_elements_by_xpath(".//div[@class='search-business-snippet-subtitle-view']")[0].text
        except: pass

        try:
            categories_elms = element.find_elements_by_xpath(".//div[@class='business-card-view__categories']/*")
            categories  = [category.text for category in categories_elms]    
        except: pass
        
        try:
            address = element.find_element_by_xpath(".//div[@class='business-card-view__address']").text
        except: pass
        
        try:
            #print('link')
            link = element.find_element_by_xpath(".//a[@class='link-wrapper']").get_attribute("href")
            #print(link)
        except: pass 

        return ImportantObject(name, rating, reviews, mean_check, categories, link,  address)
    
    
def parse_station(driver, atm):
    sections = driver.find_elements_by_xpath("//div[@class='masstransit-stops-view']//div[@class='card-dropdown-view__count']")
    lens = [int(x.text)+1 for x in sections]
    
    # раскрываем выпадающие списки
    sections = driver.find_elements_by_xpath("//div[@class='masstransit-stops-view']//div[@class='card-dropdown-view__control']")
    for x in sections[::-1]: x.click()

    sections = driver.find_elements_by_xpath("//div[@class='masstransit-stops-view']//div[@class='masstransit-stops-view__stop-name']")
    names = [x.text for x in sections] 

    sections = driver.find_elements_by_xpath("//div[@class='masstransit-stops-view']//div[@class='masstransit-stops-view__stop-distance']")
    distances = [x.text for x in sections] 
    
    if len(lens) == 1: 
        atm.bus_stations = [Station(name, distance) for name, distance in  zip(names, distances)]
    elif len(lens) == 2:
        atm.metro_stations  =  [Station(name, distance) for name, distance in list(zip(names, distances))[:lens[0]]]
        atm.bus_stations =  [Station(name, distance) for name, distance in list(zip(names, distances))[lens[0]:]]
    else:
        pass   


def clear_input(element):
    element.send_keys(Keys.CONTROL + "a");
    element.send_keys(Keys.DELETE);

def parse_organization(driver, atm):
    urli = driver.current_url
    urli = urli[:urli.rfind('=')] + '=16'
    for search_string, attr in SEARCHING_MAP.items():
        #print(search_string)
        driver.get(urli)
        #WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//input[@class='input_air-search-large__control']")))
        element = driver.find_elements_by_xpath("//input[@class='input_air-search-large__control']")[0]
        time.sleep(4)
        clear_input(element)
        ActionChains(driver).click(element).send_keys(search_string, Keys.ENTER).perform()
        time.sleep(4)
        try:
            # Парсинг элементов содержащих информацию о близлежаийших организациях  
            print('start get elements', search_string)
            elements_org = get_elements(driver)
            print('end get elements')
            # Записываем результаты в объект
            print(search_string, len(elements_org))
            result = [parse_one_organization(element) for element in elements_org] 
            setattr(atm, attr, result)
        except: 
            try:
                #print('Одна организация',search_string)
                setattr(atm, attr, [parsing_business_card_view(driver)]) 
            except:
                pass

        #element = driver.find_elements_by_xpath("//input[@class='input_air-search-large__control']")[0]
        #clear_input(element)
        #time.sleep(2)
        #ActionChains(driver).click(element).send_keys(atm.full_address, Keys.ENTER).perform()

def parse_one_organization(element):
        name, rating, reviews, mean_check, categories, link, address = [None]*7
        try:
            name = element.find_elements_by_xpath(".//div[@class='search-business-snippet-view__header']/a[1]")[0].text
        except: pass  
        
        try:
            #rating = element.find_elements_by_xpath(".//div[@class='search-business-snippet-view__rating']")[0].text
            rating = element.find_elements_by_xpath(".//span[@class='business-rating-badge-view__rating']")[0].text
        except: pass

        try:
            reviews = element.find_elements_by_xpath(".//span[@class='business-reviews-count-view']")[0].text
        except: pass

        try:
            mean_check = element.find_elements_by_xpath(".//div[@class='search-business-snippet-subtitle-view']")[0].text
        except: pass

        try:
            categories_elms = element.find_elements_by_xpath(".//div[@class='search-snippet-categories-view']/*")
            categories  = [category.text for category in categories_elms]    
        except: pass

        try:
            # print('Получаем адрес')
            address = element.find_element_by_xpath(".//div[@class='search-business-snippet-view__address']").text
            # print(address)
        except: pass 
        
        try:
            # print('link')
            link = element.find_element_by_xpath(".//a[@class='link-wrapper']").get_attribute("href")
            # print(link)
        except: pass 
        

        
        return ImportantObject(name, rating, reviews, mean_check, categories, link, address)

In [40]:
class Cell():
    """
    Объект "Банкомат"
    """
    SEARCHING_MAP = {'Еда': 'eat',
                      'Авто': 'auto',
                      'Покупки': 'purchase',
                      'Красота': 'beauty',
                      'Услуги': 'service',
                      'Развлечения и отдых': 'entertainment',
                      'Спорт': 'sport',
                      'Религия': 'religion', 
                      'Транспорт': 'transport',
                      'Госуслуги': 'government',
                      'Деньги': 'money',
                      'Образование': 'education',
                      'Стройка и ремонт': 'repair',
                      'Путешествия': 'travel'}
    
    def __init__(self, lat, long):
        self.latitude = lat 
        self.longitude = long 
        self.metro_stations = list() # Ближайшие метро и расстояния до них
        self.bus_stations = list() # Ближайшие остановки общественного транспорта и расстояния до них 
        self.inbuilding = list() # Объекты располагающиеся в том же здании, что и банкомат.
        
        self.eat = list() # Еда. Ближайшие к банкомату кафе и рестораны. Список объектов типа ImportantObject
        self.auto = list()
        self.purchase = list()
        self.beauty = list()
        self.service = list()
        self.entertainment = list()
        self.sport = list()
        self.religion = list()
        self.transport = list()
        self.government = list()
        self.money = list()
        self.education = list()
        self.repair = list()
        self.travel = list()
         
    @property
    def address(self):
        return self.address_
    
    @address.setter
    def address(self, value):
        self.address_ = value
    
    @property
    def city(self):
        return self.city_
    
    @city.setter
    def city(self, value):
        self.city_ = value
    
    def distance(self, org: Organization):
        pass    
    
    def __str__(self):
        return str([('Широта', self.latitude), ('Долгота', self.longitude)]+[(attr, len(getattr(self, attr))) for attr in  (['bus_stations','inbuilding','metro_stations'] + list(SEARCHING_MAP.values()))])
    

In [41]:
def parsing(atms, start=0):
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--window-size=1280x1024")
    driver = webdriver.Chrome('chromedriver.exe', chrome_options=chrome_options)
    driver.maximize_window()
    url = 'https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text='
    for i, atm in  tqdm_notebook(enumerate(atms[start:])):
        #urli = url + ' '.join([atm.address, atm.city]).replace(' ', '%20')
        urli = url + str(atm.latitude) +', ' + str(atm.longitude)
        # print(urli)
        driver.get(urli)
        element = driver.find_elements_by_xpath("//input[@class='input_air-search-large__control']")[0]
        ActionChains(driver).click(element).send_keys(Keys.ENTER).perform()
        time.sleep(10)
        #driver.find_element_by_xpath("//div[@class='small-search-form-view__button//button']").click()
        try:             
            # Прасинг ближайших станций метро и автобусных остановок
            try:
                parse_station(driver, atm)
            except:
                pass

            # Парсинг организаций находящихся в здании 
            try:
                parse_inbuilding(driver, atm)
            except: pass
                       
            # Парсинг организаций оказывающих услуги в различных сферах
            try:
                parse_organization(driver, atm)
            except: pass
            
            # Сохраняем и распечатываем результаты
            pickle.dump(atm, open('D:/Moscow/cell_%s_%s.dat'%(atm.latitude, atm.longitude), 'wb'))
            print(str([('Широта', atm.latitude), ('Долгота', atm.longitude)]+[(attr, len(getattr(atm, attr))) for attr in  (['bus_stations','inbuilding','metro_stations'] + list(SEARCHING_MAP.values()))]))
        except:
            print(i,' fail')
            pass

In [42]:
def get_elements(driver):
        # Парсинг элементов списка содержащих информацию о близлежаийших организациях 
        elements = list()
        i = 0
        while len(driver.find_elements_by_xpath("//div[@class='search-list-view__add-business']")) == 0  or i < 50:
            elements = driver.find_elements_by_xpath("//div[@class='search-list-view__list']//li")
            ActionChains(driver).move_to_element(elements[-1]).perform() # move_to_element работает только когда мышки нет окне браузера
            time.sleep(0.5)
            i+=1
        elements = driver.find_elements_by_xpath("//div[@class='search-list-view__list']//li")
        # print(len(elements))
        return  elements 

In [4]:
from itertools import product

In [5]:
latitudes = 55.38 + 0.009*np.array(range(1,71))

In [6]:
longitudes = 37.04 + 0.016*np.array(range(1,65))

In [7]:
cells = [Cell(lat, long) for lat, long in product(latitudes, longitudes)]

NameError: name 'Cell' is not defined

In [9]:
import os
l= os.listdir('D:/Moscow/')

c = 0
for lat, long in [(lat, long) for lat, long in product(latitudes, longitudes)][:2200]:
    c+=1 
    if 'cell_%s_%s.dat'%(lat, long) not in l:
        print(c, lat, long)

125 55.398 38.016
126 55.398 38.032
210 55.416000000000004 37.327999999999996
211 55.416000000000004 37.344
260 55.425000000000004 37.104
530 55.461000000000006 37.327999999999996
531 55.461000000000006 37.344
743 55.488 37.664
744 55.488 37.68
896 55.506 38.064
1036 55.533 37.232
1109 55.542 37.376
1210 55.551 37.967999999999996
1216 55.551 38.064
1326 55.569 37.775999999999996
1416 55.587 37.168
1443 55.587 37.6
1781 55.632000000000005 37.888
1782 55.632000000000005 37.903999999999996
1783 55.632000000000005 37.92
1784 55.632000000000005 37.936
1785 55.632000000000005 37.952
1786 55.632000000000005 37.967999999999996
1787 55.632000000000005 37.984
1788 55.632000000000005 38.0
1789 55.632000000000005 38.016
1790 55.632000000000005 38.032
1791 55.632000000000005 38.048
1792 55.632000000000005 38.064
1793 55.641000000000005 37.056
1794 55.641000000000005 37.071999999999996
1795 55.641000000000005 37.088
1796 55.641000000000005 37.104
1797 55.641000000000005 37.12
1798 55.641000000000005

In [43]:
parsing(cells[260:400])

start get elements Еда
end get elements
Еда 29
start get elements Авто
end get elements
Авто 37
start get elements Покупки
end get elements
Покупки 74
start get elements Красота
end get elements
Красота 40
start get elements Услуги
end get elements
Услуги 28
start get elements Развлечения и отдых
end get elements
Развлечения и отдых 20
start get elements Спорт
end get elements
Спорт 30
start get elements Религия
end get elements
Религия 2
start get elements Транспорт
end get elements
Транспорт 115
start get elements Госуслуги
end get elements
Госуслуги 14
start get elements Деньги
end get elements
Деньги 34
start get elements Образование
end get elements
Образование 359
start get elements Стройка и ремонт
end get elements
Стройка и ремонт 85
start get elements Путешествия
end get elements
Путешествия 8
0  fail
start get elements Еда
end get elements
Еда 23
start get elements Авто
end get elements
Авто 14
start get elements Покупки
end get elements
Покупки 3
start get elements Красота
e